In [ ]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()


def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    prefix = "[image_data_exist_on_disk]"
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"{prefix} {image.path} contains data")
        else:
            print(f"{prefix} {image.path} does *not* contain data")
            all_images_have_data = False
    return all_images_have_data

In [ ]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

In [ ]:
# Run create_ome_zarr+yokogawa_to_zarr
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        ),
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Print current dataset information
debug(dataset);

In [ ]:
# Run init_channel_parallelization
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction_compound"],
            args_non_parallel=dict(overwrite_input=False),
        ),
    ],
    dataset=dataset,
    executor=executor,
)

# Check that custom parallelization_list
assert len(dataset.images) == 4

for image in dataset.images[0:2]:
    assert image.flags.get("illumination_correction") is None
for image in dataset.images[2:4]:
    assert image.flags.get("illumination_correction") is True

# Print current dataset information
debug(dataset);

In [ ]:
executor.shutdown()